# Introduction aux modèles d'arbres

_Objectif du cours_:
  - savoir ce qu'est un *arbre de décision*
  - savoir en construire un à la main en partant d'un petit jeu de données grâce à l'algorithme basé sur le gain d'information par entropie
  - savoir en construire un en utilisant une solution clé en mains de `scikit-learn`. Savoir interpréter les résultats

# Sommaire

* [0. Jeu préparatoire: Qui est-ce?](#Jeu_preparatoire)
* [1. Définition d'un arbre de décision](#Definition_arbre_de_decision)
* [2. Comment mesurer l'information? Modèle d'entropie de Claude Shannon](#Entropy_model)
  * [2.0. Exercice préparatoire : exemple de la détection de mail spam](#Entropy_model)
  * [2.1. Définition de l'entropie](#Entropy_model)
    * [2.1.0. Intuition pour un jeu de cartes](#Entropy_model)
    * [2.1.1. Formule mathématique](#Formula)
    * [2.1.2. Représentation de la courbe pour le cas binaire](#Binary_case)
  * [2.2. Gain d'information](#Gain_Information)
    * [2.2.0. Exercice](#Gain_Information)
    * [2.2.0. Algorithme](#Algorithme)
      * [Exercice 1: cas de variables binaires](#Algorithme)
      * [Exercice 2: cas de variables non binaires](#Algorithme)
* [3. Forêts aléatoires](#Forets)
  * [3.0. Définition et Algorithme](#Forets)
  * [3.1. Exercice](#Forets)

<a name="Jeu_preparatoire"></a>
## 0. Jeu préparatoire: `Qui est-ce?`

On va commencer par un jeu `Qui est-ce?`. Les règles du jeu sont:
- 2 joueurs
- 1 joueur choisit une carte avec l'image d'un personnage dans un paquet de cartes:

![people_guess_who.png](../images/people_guess_who.png)

qui peuvent être représentées numériquement par le tableau:

| Est-un homme? | As de long cheveux? | Porte des lunettes? | Nom  |
|-----|-----------|---------|-------|
| Oui | Non        | Oui     | Brian |
| Oui | Non        | Non      | John  |
| Non  | Oui       | Non      | Aphra |
| Non  | Non        | Non      | Aoife |

- L'autre joueur :
  * essaie de deviner quel personnage est sur la carte en posant une série de questions. La réponse est binaire c'est-à-dire que l'adversaire répond seulement par `oui` ou `non`.
  * il gagne en devinant qui est sur la carte dans un `petit nombre de questions` et perd sinon


_Exemple d'une série de questions commençant par `La personne porte-t-elle des lunettes?`:_

![does_the_person_wear_glasses.png](../images/does_the_person_wear_glasses.png)

1. En commençant par cette question, quel est le nombre moyen de questions posées par partie?

Quel est l'arbre de décision optimal?

![optimal_tree_guess_who.png](../images/optimal_tree_guess_who.png)
2. Reprendre le jeu avec cette série de questions. Quel est le nombre moyen de questions posées par partie ?

<a name="Definition_arbre_de_decision"></a>
## 1. Définition d'un arbre de décision

Terminologie:

![Decision-Tree terminology](../images/decision_tree_nodes.png)

* `racine`: ...
* `noeud interne`: ...
* `feuille`: ...

<a name="Entropy_model"></a>
## 2. Comment mesurer l'information? Modèle d'`entropie` de _Claude Shannon_

Comment capturer mathématiquement l'*information contenue dans une variable descriptive*?
### 2.0. Exercice préparatoire : exemple de la `détection de mail spam`

| ID  |MOTS SUSPECTS | EXPEDITEUR INCONNU | CONTIENT DES IMAGES | CATEGORIE |
|-----|-------|--------|--------|----------|
| 376 | Vrai  | Faux  | Vrai   | spam     |
| 489 | Vrai  | Vrai   | Faux  | spam     |
| 541 | Vrai  | Vrai   | Faux  | spam     |
| 693 | Faux | Vrai   | Vrai   | normal   |
| 782 | Faux | Faux  | Faux  | normal   |
| 976 | Faux | Faux  | Faux  | normal   |

Le jeu de données ci-dessus contient 3 variables descriptives binaires:
* `MOTS SUSPECTS` est vrai si un courriel contient un ou plusieurs mots que l'on trouve généralement dans les e-mails de spam (par exemple, casino, viagra, banque ou compte)
* `EXPEDITEUR INCONNU` est vrai si l'e-mail provient d'une adresse qui n'est pas répertoriée dans les
dans les contacts de la personne qui a reçu l'e-mail
* `CONTIENT DES IMAGES` est vrai si l'e-mail contient une ou plusieurs images. vrai si l'e-mail contient une ou plusieurs images.

Exemples d'arbres de décision cohérents avec l'échantillon (expliquez pourquoi):

![spam_detection_example_of_decision_trees.png](../images/spam_detection_example_of_decision_trees.png)

### 2.1. Définition de l'entropie

C'est une mesure informatique de l'impureté d'un ensemble.

#### 2.1.0. Intuition pour un jeu de cartes
C'est l'incertitude associée au fait de deviner le résultat d'une sélection aléatoire dans le jeu. Par exemple, 
* si vous deviez sélectionner au hasard une dans l'ensemble de la figure (a), votre incertitude serait nulle (Pourquoi? R/ car vous seriez certain de choisir une carte.) Cet ensemble présente donc
zéro entropie. 
* Si, par contre, vous deviez choisir au hasard un élément de l'ensemble vous seriez très incertain quant à toute prédiction, (Pourquoi?) C'est pourquoi cet ensemble a une entropie très élevée.

![entropie_jeu_de_cartes.png](../images/entropie_jeu_de_cartes.png)


In [ ]:
import numpy as np
from summer.model.tree import entropy_for_binary_variable, entropy
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
FIGSIZE = (15, 5)
VERBOSE = True

In [ ]:
entropie_jeu_de_cartes_a = entropy_for_binary_variable(1)
np.testing.assert_allclose(entropie_jeu_de_cartes_a, 0)

probabilities = [1/12 for i in range(0, 12)]
entropie_jeu_de_cartes_f = entropy(probabilities)
np.testing.assert_allclose(entropie_jeu_de_cartes_f, 3.585, atol=1e-3)

_Question_: Calculer l'entropie du jeu (e)

<a name="Formula"></a>
#### 2.1.1. Formule mathématique

L'équation suivante est la pierre angulaire de la théorie moderne de l'information et est une excellente mesure de l'impureté càd de l'hétérogénéité, d'un ensemble, d'une population.
\begin{equation}
H(t)  = -\sum_{i = 1}^{l} P(t=i) * log_{s}(P(t=i))
\end{equation}
où:
* $t$ = "target feature", la variable cible
* $i$ = une catégorie de la variable cible
* $P(t=i)$ est la probabilité que le résultat de la sélection aléatoire d'un élément $t$ soit du type $i$.
* $l$ est le nombre de types différents de choses dans l'ensemble,
* $s$ est une base logarithmique arbitraire. On la prend  historiquement égale à 2, pour obtenir un résultat en nombre de bits.

Quelle est l'intuition derrière la formule?

<a name="Binary_case"></a>
#### 2.1.2. Représentation de la courbe pour le cas binaire
\begin{align*}
p &= P(t=1) \\
H(t)  &= - p * log_{2}(p) - (1 - p) * log_{2}(1 - p)
\end{align*}


In [ ]:
min_probability, max_probability = 0, 1
n_steps = 100
probabilities = np.linspace(min_probability, max_probability, n_steps)
entropies = [
    entropy_for_binary_variable(probability) for probability in probabilities
]

In [ ]:
opt_entropy_coordinates = {"x": 0.5, "y": 1}
kwargs = dict(linestyle="--", color="red")
if VERBOSE:
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax.scatter(x=probabilities, y=entropies)
    ax.axvline(opt_entropy_coordinates["x"], **kwargs)
    ax.axhline(opt_entropy_coordinates["y"], **kwargs)
    plt.show()

<a name="Gain_Information"></a>
## 2.2. Gain d'information
_Quelle est la relation entre une mesure de l'hétérogénéité d'un ensemble et l'analyse prédictive ?_ 

Si on peut construire une séquence de tests qui divise les données d'apprentissage en ensembles purs par rapport aux valeurs des caractéristiques cibles, nous pouvons alors étiqueter un individu en 
* lui appliquant la même séquence de tests et en 
* l'étiquetant avec la valeur de la caractéristique cible des instances dans l'ensemble où elle se trouve.

#### 2.2.1. Exercice
Revenons au jeu de données d'emails. Quelle 1ere variable choisir pour patitionner? 

Indications: 
* C'est la variable qui discriminerait le mieux entre un spam et un mail normal. 
* Voici comment les instances de l'ensemble de données de spam se répartissent lorsque nous utilisons chacune des variables descriptives de l'ensemble de données de spam.

![entropie_jeu_de_cartes.png](../images/feature_selection.png)

Modèle (lien intuition - entropie): c'est la variable qui réduit le plus l'entropie globale 


<a name="Algorithme"></a>
#### 2.2.1. Algorithme
* 1. Calculer l'entropie de l'ensemble de données original par rapport à la caractéristique cible. Cela nous donne une mesure de la quantité d'information nécessaire pour organiser l'ensemble de données en ensembles purs.
\begin{align*}
H(t, \mathcal{D})  &= -\sum_{l \in levels(t)} P(t=i) * log_{s}(P(t=i)) \\
\end{align*}
* 2. Pour chaque caractéristique descriptive, créez les ensembles qui résultent de la partition des instances de l'ensemble de données en utilisant leurs caractéristiques. Puis additionnez les scores d'entropie de chacun de ces ensembles. Cela donne une mesure de l'information qui
nécessaires pour organiser les instances en ensembles purs après les avoir divisées à l'aide de la caractéristique descriptive.
\begin{align*}
rem(d, \mathcal{D}) &= \sum_{l \in levels(t)} \frac{|\mathcal{D}_{d=l}|}{|\mathcal{D}|} * H(t, \mathcal{D}_{d=l}) \\
\end{align*}
* 3. Soustraire la valeur d'entropie restante (calculée à l'étape 2) de la valeur d'entropie
d'origine (calculée à l'étape 1) pour obtenir le gain d'information.
\begin{align*}
IG(d, \mathcal{D})  &= H(t, \mathcal{D}) - rem(d, \mathcal{D}) 
\end{align*}

#### Exercice 1: cas variables binaires
Implémenter cet algorithme sur le dataset spam.

#### Exercice 2: cas variables non binaires
Implémenter cet algorithme sur un dataset de végétation.

| ID | STREAM | SLOPE    | ELEVATION | VEGETATION |
|----|--------|----------|-----------|------------|
| 1  | FALSE  | steep    | high      | chapparal  |
| 2  | TRUE   | moderate | low       | riparian   |
| 3  | TRUE   | steep    | medium    | riparian   |
| 4  | FALSE  | steep    | medium    | chapparal  |
| 5  | FALSE  | flat     | high      | conifer    |
| 6  | TRUE   | steep    | highest   | conifer    |
| 7  | TRUE   | steep    | high      | chapparal  |


Quel est l'arbre inféré?

R/ 
![final_tree_for_vegetation_dataset.png](../images/final_tree_for_vegetation_dataset.png)

Quelle prédiction aurait-on pour cet individu? 

| ID | STREAM | SLOPE    | ELEVATION | VEGETATION |
|----|--------|----------|-----------|------------|
| 8  | TRUE   | moderate | high      | chapparal  |

_Remarque_: il n'est pas dans l'ensemble d'apprentissage. Donc nous généralisons la décision, au-delà du jeu d'apprentissage.

## Exercice 3
Comparer avec le résultat donné par `scikit-learn`

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix,
    plot_confusion_matrix, 
    classification_report
)

from summer.preprocessing.encoding import encode_ordinal
from summer.tools.common_path import ROOT_PATH

from cloud_io.gcp.io import download_file

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
TARGET_COL = "VEGETATION"
FEATURES_COLS = ["STREAM", "SLOPE", "ELEVATION"]

In [ ]:
BUCKET_NAME = "decision_tree_course"
blob_name = "data/vegetation.csv"
download_path = Path(ROOT_PATH, blob_name)

# Download file

download_path.unlink(missing_ok=True)
assert not download_path.exists()

DOWNLOAD_KWARGS = dict(
    bucket_as_local=ROOT_PATH,
    bucket_name=BUCKET_NAME,
)
download_path_out = download_file(download_path, **DOWNLOAD_KWARGS)

# Read file
df = pd.read_csv(
    download_path_out,
    index_col="ID"
)

In [ ]:
expected_shape = (7, 4)
np.testing.assert_allclose(df.shape, expected_shape)

In [ ]:
X, y = df[FEATURES_COLS], df[TARGET_COL]

In [ ]:
categories = {
    "STREAM": {False: 0, True: 1},
    "SLOPE": {"flat": 0, "moderate": 1, "steep": 2,},
    "ELEVATION": {"low": 0, "medium": 1, "high": 2, "highest": 3},
}
X_preprocessed = encode_ordinal(X, categories)

In [ ]:
tree_kwargs = dict(random_state=0)
classifier = DecisionTreeClassifier(criterion="entropy", **tree_kwargs)

In [ ]:
classifier.fit(X_preprocessed, y)

In [ ]:
if VERBOSE:
    fig, ax = plt.subplots(1, 1, figsize=FIGSIZE)
    plot_tree(classifier, feature_names=FEATURES_COLS, ax=ax) 
    plt.show()

<a name="Forets"></a>
## 3. Forêts aléatoires

### 3.0. Définition et Algorithme
* 1. Bagging = **B**ootstrap **Agg**regat**ing** avec des arbres de décisions: 
  * chaque modèle de l'ensemble est entraîné sur un échantillon aléatoire de l'ensemble de données.
ensemble est entraîné sur un _échantillon aléatoire de l'ensemble de données_. Important! chaque échantillon aléatoire est de la même taille que l'ensemble de données et l'échantillonnage avec remplacement ou **tirage avec remise** est utilisé. On appelle ces échantillons aléatoires, _échantillons bootstrap_. 
  * un modèle est induit à partir de chaque échantillon bootstrap. Donc ils sont tous différents.
* 2. Échantillonnage du sous-espace des variables: l'échantillonnage est étendu de manière à ce que chaque échantillon bootstrap n'utilise qu'un sous-ensemble aléatoire des variables. Cet échantillonnage de l'ensemble des caractéristiques encourage davantage la diversité des arbres au sein de l'ensemble.
ensemble et présente l'avantage de réduire le temps d'apprentissage de chaque arbre.
* 3. la prédiction finale s'obtient en par un **vote à la majorité**



### 3.1. Exercice
Le tableau suivant présente un jeu de données contenant les détails de 5 participants à une étude sur les maladies cardiaques, et une caractéristique cible `RISQUE DE MALADIE DU COEUR` qui décrit leur risque de maladie cardiaque. Chaque patient est décrit en fonction de 5 caractéristiques descriptives binaires :
* `EXERCICE PHYSIQUE` : à quelle fréquence fait-il de l'exercice ?
* `FUMEUR` : fume-t-il ?
* `OBESE` : est-il en surpoids ?
* `FAMILLE`: est-ce que l'un de leurs parents ou frères et soeurs a souffert d'une maladie cardiaque ?

| ID | EXERCICE PHYSIQUE | FUMEUR | OBESE | FAMILLE | RISQUE DE MALADIE DU COEUR |
|----|----------|--------|-------|--------|---------------------|
| 1  | journalier    | Faux  | Faux | Oui    | faible                 |
| 2  | hebdomadaire   | Vrai   | Faux | Oui    | élevé                |
| 3  | journalier    | Faux  | Faux | Non     | faible                 |
| 4  | rare   | Vrai   | Vrai  | Oui    | élevé                |
| 5  | rare   | Vrai   | Vrai  | Non     | élevé                |

a) Dans le cadre de l'étude, les chercheurs ont décidé de créer un modèle prédictif pour sélectionner les participants, en fonction de leur risque de maladie cardiaque. Il vous a été demandé d'implémenter ce modèle de dépistage en utilisant une **Forêt aléatoire**.

Les 3 tableaux ci-dessous répertorient 3 échantillons bootstrap qui ont été générés à partir de l'ensemble de données ci-dessus. En utilisant ces échantillons bootstrap, créez les arbres de décision qui seront dans le modèle Forêt aléatoire (utilisez le gain d'information basé sur l'entropie comme critère de sélection des variables).

| ID | EXERCICE PHYSIQUE | FAMILLE | RISQUE DE MALADIE DU COEUR |
|----|----------|--------|---------------------|
| 1  | journalier    | Oui    | faible                 |
| 2  | hebdomadaire   | Oui    | élevé                |
| 3  | journalier    | Non     | faible                 |
| 4  | rare   | Oui    | élevé                |
| 5  | rare   | Non     | élevé                |

| ID | EXERCICE PHYSIQUE| OBESE | RISQUE DE MALADIE DU COEUR |
|----|--------|-------|---------------------|
| 1  | Faux  | Faux | faible                 |
| 2  | Vrai   | Faux | élevé                |
| 3  | Faux  | Faux | faible                 |
| 4  | Vrai   | Vrai  | élevé                |
| 5  | Vrai   | Vrai  | élevé                |

| ID | OBESE | FAMILLE | RISQUE DE MALADIE DU COEUR |
|----|-------|--------|---------------------|
| 1  | Faux | Oui    | faible                 |
| 2  | Faux | Oui    | élevé                |
| 3  | Faux | Non     | faible                 |
| 4  | Vrai  | Oui    | élevé                |
| 5  | Vrai  | Non     | élevé                |

b) En supposant que le modèle RF que vous avez créé utilise le vote majoritaire, quelle prédiction donnera-t-il pour la requête suivante ?

| ID | EXERCICE PHYSIQUE | FUMEUR | OBESE | FAMILLE |
|----|----------|--------|-------|--------|
| 1  | rare    | Faux  | Vrai | Oui    |


## Références
1. _Fundamentals of Machine Learning for Predictive Data Analytics: Algorithms, Worked Examples, and Case Studies_, Book by Aoife D'Arcy, Brian Mac Namee, and John D. Kelleher. Chapter 4 Information-based Learning. 
  * _Lien web pour extrait Arbres_:
https://machinelearningbook.com/wp-content/uploads/2015/07/FMLPDA_SampleChapter_InformationBasedLearning.pdf
  * _Livre en entier_: https://dokumen.pub/qdownload/fundamentals-of-machine-learning-for-predictive-data-analytics-algorithms-worked-examples-and-case-studies-1-edition-july-24-2015-978-0262029445.html
2. Solutions: http://machinelearningbook.com/wp-content/uploads/2015/07/FMLPDA_freely_avail_solutions.pdf

---
# End of script
---